In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from torchvision.utils import DataLoader

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,32,5)
        self.conv2 = nn.Conv2d(32,64,5)
        self.conv3 = nn.Conv2d(64,128,5)
        
        x = torch.randn(48,48).view(-1,1,48,48)
        self._to_linear = None
        self.convs(x)
#         self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512,2)
        
    def convs(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))
        
        print(x[0].shape)
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    
    def forward(self,x):
        x = self.convs(x)
        x = x.view(-1,self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x,dim=1)

In [8]:
cnn = CNN()

torch.Size([128, 2, 2])


In [ ]:
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1,48,48)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1
val_size = int(len(X)*VAL_PCT)
print(val_size)

In [ ]:
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

print(len(train_X))
print(len(test_X))

In [ ]:
BATCH_SIZE = 32
EPOCHS = 1

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
#         print(i, i+BATCH_SIZE)
        batch_X = train_X[i:i+BATCH_SIZE].view(-1,1,48,48)
        batch_y = train_y[i:i+BATCH_SIZE]
        
        cnn.zero_grad()
        outputs = cnn(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backwards()
        optimizer.step()

print(loss)
    

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        cnn_out = cnn(test_X[i].view(-1,1-48,48))[0]
        
#         print(real_class, cnn_out)
        predicted_class = torch.argmax(cnn_out)
        if predicted_class == real_class:
            correct+=1
        total+=1
print('Accuracy: ', round(correct/total,3))